In [10]:
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import torch.nn as nn
# 假设你有不等长的序列数据集
data = [
    torch.tensor([1, 2, 3], dtype=torch.float32),
    torch.tensor([4, 5], dtype=torch.float32),
    torch.tensor([6, 7, 8, 9], dtype=torch.float32),
]


In [11]:
# 填充序列到相同长度
padded_data = pad_sequence(data, batch_first=True, padding_value=0)
# 打包填充后的序列
packed_data = pack_padded_sequence(padded_data, lengths=[len(seq) for seq in data], batch_first=True, enforce_sorted=False)


In [20]:
print(padded_data.unsqueeze(0))
print(padded_data.unsqueeze(0).shape)
print(packed_data)
padded_data = padded_data.unsqueeze(0)

tensor([[[1., 2., 3., 0.],
         [4., 5., 0., 0.],
         [6., 7., 8., 9.]]])
torch.Size([1, 3, 4])
PackedSequence(data=tensor([6., 1., 4., 7., 2., 5., 8., 3., 9.]), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))


In [24]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

# 假设你有不等长的序列数据集
data = [
    torch.tensor([1, 2, 3], dtype=torch.float32),
    torch.tensor([4, 5], dtype=torch.float32),
    torch.tensor([6, 7, 8, 9], dtype=torch.float32),
]


# 填充序列到相同长度
padded_data = pad_sequence(data, batch_first=True, padding_value=0)

# 打包填充后的序列
packed_data = pack_padded_sequence(padded_data, lengths=[len(seq) for seq in data], batch_first=True, enforce_sorted=False)

class GRU_structure(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU_structure, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        # 打包后的数据需要在forward方法中进行解包处理
        packed_output, hidden = self.gru(x)
        return packed_output, hidden

# 实例化模型
input_size = 1  # 假设输入特征大小
hidden_size = 10  # 假设隐藏层大小
model = GRU_structure(input_size, hidden_size)

# 将打包后的数据输入模型
output, hidden = model(packed_data)

# 输出打包后的数据形状
print(f"Packed output shape: {output.data.shape}")
print(f"Hidden shape: {hidden.shape}")


RuntimeError: input must have 2 dimensions, got 1

In [25]:

import torch as t
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

a = t.tensor([[1,2,3],[6,0,0],[4,5,0],[1,1,0]]) #(batch_size, max_length)
lengths = t.tensor([3,1,2,2])

# 排序
a_lengths, idx = lengths.sort(0, descending=True)
_, un_idx = t.sort(idx, dim=0)
a = a[idx]

# 定义层 
emb = t.nn.Embedding(20,2,padding_idx=0) 
lstm = t.nn.LSTM(input_size=2, hidden_size=4, batch_first=True) 

# print(a)
a_input = emb(a)
print(a_input.size())
# print(a_lengths)
a_packed_input = t.nn.utils.rnn.pack_padded_sequence(input=a_input, lengths=a_lengths, batch_first=True)




print(a_packed_input)
packed_out, _ = lstm(a_packed_input)
out, _ = pad_packed_sequence(packed_out, batch_first=True)



torch.Size([4, 3, 2])
PackedSequence(data=tensor([[-1.3637, -2.4568],
        [ 0.0541,  0.1065],
        [-1.3637, -2.4568],
        [-1.1491,  1.0280],
        [-1.0348, -0.9046],
        [-0.8591,  0.2043],
        [-1.3637, -2.4568],
        [-0.4978, -0.2117]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([4, 3, 1]), sorted_indices=None, unsorted_indices=None)


In [28]:
print(out.shape)
print(_.shape)

torch.Size([4, 3, 4])
torch.Size([4])


In [36]:
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import DataLoader
import torch

# 定义模型参数
input_size = 1  # 输入特征的数量
hidden_size = 20  # 隐藏层的大小
num_layers = 2  # GRU层的数量

# 定义GRU模型
gru_model = torch.nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
# 假设你的数据是这样的
# 假设你的数据是这样的
data = [torch.tensor([1,2,3], dtype=torch.float32).unsqueeze(-1), 
        torch.tensor([4,5], dtype=torch.float32).unsqueeze(-1),
        torch.tensor([6], dtype=torch.float32).unsqueeze(-1)]

# 使用pad_sequence进行填充
data = pad_sequence(data, batch_first=True)

# 创建DataLoader
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    lengths = [len(seq) for seq in data]
    data = pad_sequence(data, batch_first=True)
    return data, lengths

dataloader = DataLoader(data, batch_size=32, collate_fn=collate_fn)

# 在训练循环中
for batch, lengths in dataloader:
    batch = pack_padded_sequence(batch, lengths, batch_first=True)
    output, hidden = gru_model(batch)
    print(output)
    print(hidden.shape)

PackedSequence(data=tensor([[ 0.1030,  0.0411, -0.0596,  0.0050,  0.1488,  0.0403,  0.0464,  0.0117,
          0.0133, -0.0384, -0.0556, -0.0670, -0.1021, -0.0424,  0.1172,  0.1525,
          0.0574,  0.0087, -0.0549,  0.1167],
        [ 0.0401,  0.0577, -0.0427,  0.0531,  0.1620,  0.0675,  0.0359, -0.0290,
          0.0380,  0.0388, -0.0047, -0.0710, -0.0959, -0.0398,  0.1403,  0.1600,
          0.1422, -0.0233, -0.1167,  0.1818],
        [ 0.0022,  0.0510, -0.0369,  0.0731,  0.1526,  0.0692,  0.0256, -0.0527,
          0.0430,  0.0541,  0.0144, -0.0595, -0.0840, -0.0529,  0.1381,  0.1475,
          0.1716, -0.0313, -0.1550,  0.2148],
        [ 0.1260,  0.1094, -0.0887,  0.0345,  0.2670,  0.0928,  0.0793,  0.0151,
          0.0183,  0.0048, -0.0727, -0.1323, -0.1641, -0.0354,  0.1459,  0.2644,
          0.1268, -0.0038, -0.1122,  0.2000],
        [-0.0060,  0.1404, -0.0539,  0.1457,  0.2829,  0.1409,  0.0683, -0.0656,
          0.0507,  0.1389,  0.0198, -0.1369, -0.1490, -0.0254,  0.1

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# 独热编码函数
def one_hot_encode(sequence, max_len, vocab_size):
    encoding_map = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot_encoded = np.zeros((max_len, vocab_size), dtype=int)

    for i, base in enumerate(sequence):
        if i >= max_len:
            break
        index = encoding_map.get(base, -1)
        if index != -1:
            one_hot_encoded[i, index] = 1

    return one_hot_encoded

# 自定义数据集类
class GeneDataset(Dataset):
    def __init__(self, file_path, max_len, vocab_size):
        self.data = pd.read_csv(file_path, header=None)
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.data['encoded_sequence'] = self.data[1].apply(lambda seq: one_hot_encode(seq, max_len, vocab_size))
        self.data['length'] = self.data[1].apply(len)
        self.labels = torch.tensor(self.data[2].values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        encoded_sequence = self.data.iloc[idx]['encoded_sequence']
        length = self.data.iloc[idx]['length']
        label = self.labels[idx]
        return torch.tensor(encoded_sequence, dtype=torch.float32), torch.tensor(length), label

# 生成数据加载器的批处理函数
def collate_fn(batch):
    sequences, lengths, labels = zip(*batch)

    # 转换为tensor
    sequences = [torch.tensor(seq) for seq in sequences]
    lengths = torch.tensor(lengths, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.long)

    # 填充序列
    padded_sequences = pad_sequence(sequences, batch_first=True)

    # 打包序列
    packed_sequences = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
    return packed_sequences, lengths, labels

# 定义简单的GRU模型
class SimpleGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, packed_input):
        packed_output, _ = self.gru(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.fc(output[:, -1, :])  # 取最后一个时间步的输出
        return output

# 训练模型的函数
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for packed_sequences, lengths, labels in dataloader:
            print(packed_sequences)
            print(labels)
            outputs = model(packed_sequences)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# 测试数据加载器和GRU模型
def main():
    # 参数设置
    file_path = '/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/CNN/测试代码数据集.csv'  # 替换为你的文件路径
    max_len = 500  # 最大序列长度
    vocab_size = 4  # 独热编码的维度 (A, C, G, T)
    input_size = vocab_size  # 输入特征维度
    hidden_size = 128  # 隐藏层维度
    output_size = 2  # 输出类别数量 (假设二分类)
    batch_size = 32  # 批次大小
    num_epochs = 10  # 训练轮数

    # 创建数据集和数据加载器
    dataset = GeneDataset(file_path, max_len, vocab_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # 定义模型、损失函数和优化器
    model = SimpleGRU(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 训练模型
    train_model(model, dataloader, criterion, optimizer, num_epochs)

    # 测试模型
    model.eval()
    with torch.no_grad():
        for packed_sequences, lengths, labels in dataloader:
            outputs = model(packed_sequences)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == labels).sum().item() / labels.size(0)
            print(f'Accuracy: {accuracy:.4f}')
            break

if __name__ == "__main__":
    main()


/tmp/ipykernel_3527421/2604154938.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequences = [torch.tensor(seq) for seq in sequences]


PackedSequence(data=tensor([[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0